In [1]:
import requests
from bs4 import BeautifulSoup
import datetime
import pymysql
pymysql.install_as_MySQLdb()
import time

In [2]:
def get_page(link):
    header = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'}
    response = requests.get(link, headers=header).content
    html = response.decode('UTF-8')
    soup = BeautifulSoup(html,'lxml')
    return soup

In [3]:
def get_data(post_list):
    data_list=[]
    for post in post_list:
        title_id = post.find('div',class_='post-title')
        title = title_id.a.text.strip()
        post_link = title_id.a['href']
        post_link = 'https://bbs.hupu.com'+post_link
        
        #作者、创建时间
        author_div = post.find('div',class_='post-auth')
        date_div = post.find('div',class_='post-time')
        author = author_div.a.text.strip()
        start_data = date_div.get_text()
        
        #回复人数、浏览次数
        reply_view = post.find('div',class_='post-datum').text.strip()
        reply = reply_view.split('/')[0].strip()
        view = reply_view.split('/')[1].strip()
        
        data_list.append([title,post_link,author,start_data,reply,view])
    return data_list

In [4]:
class MysqlAPI(object):
    def __init__(self,db_ip,db_user,db_password,db_name,db_charset):
        self.db_ip = db_ip
        self.db_user = db_user
        self.db_password = db_password
        self.db_name = db_name
        self.db_charset = db_charset
        self.conn = pymysql.connect(host=self.db_ip,user=self.db_user,
                                    password=self.db_password,db=self.db_name,charset=self.db_charset)
        self.cur = self.conn.cursor()
    def add(self,title,post_link,author,start_data,reply,view):
        sql = "INSERT IGNORE INTO table (column_list)"
        sql = "INSERT INTO hupu(title,post_link,author,start_data,reply,view)"\
                                   "VALUES (\'%s\',\'%s\',\'%s\',\'%s\',\'%s\',\'%s\')"
        self.cur.execute(sql %(title,post_link,author,start_data,reply,view))
        self.conn.commit()

In [ ]:
for i in range(1,5):
    link = 'http://bbs.hupu.com/502-'+str(i)
    print('开始第%s页数据爬取...' %i)
    soup = get_page(link)
    soup = soup.find('div',class_='bbs-sl-web-post')
    post_list = soup.find_all('li')
    data_list = get_data(post_list)
    hupu_post = MysqlAPI('localhost','root','980211','mysql','utf8')
    for each in data_list:
        hupu_post.add(each[0],each[1],each[2],each[3],each[4],each[5])
    print('第%s页爬取完成！' %i)
    time.sleep(3)